# Quantifying catastrophic forgetting in split sequences
#### Simon Lebastard - 02/07/2020

In [1]:
%matplotlib widget

In [2]:
import os

project_root = '/home/proprietaire/Documents/Workspace/Jobs/Columbia/ultrametric_benchmark/Ultrametric-benchmark'

In [3]:
os.chdir(project_root)
import result_loader as ld

In [4]:
seq_length = 200000
n_batches = 10
lr=0.05

# Foar artificial ultrametric dataset only
linear_ratio_for_artificial_seq = 8
artificial_seq_len = 200

In [5]:
import ast

with open('Results/simu_mapping_dict.txt', 'r', encoding='utf-8') as filenames:
    filenames_dct_txt = filenames.read().replace('\n', '')
    
datapaths = ast.literal_eval(filenames_dct_txt)

In this notebook, we'll loop through the files produced by the ultrametric framework accross temperatures and shuffle block size, and construct dictionnaries indexed by [T, blocksz].
We will then use those dicts to create the plots for DARPA.

Now let's load standard packages and find out about the content of each file:

In [6]:
import numpy as np
import pandas as pd
from mpl_toolkits import mplot3d
from matplotlib import pyplot as plt
import pdb

import pickle

In [7]:
os.chdir(project_root+'/Results')

## Random split (random two-split) scenario - Various lengths ## 

dataset= 'artificial_32'
artificial_seq_len = 200
linear_ratio_for_artificial_seq = 8

rs_rb2 = {}

for hidden_sz in [5, 10, 20, 60, 200]:
    dataroot = project_root+'/Results/1toM/' + dataset + '/' + 'FCL'+str(hidden_sz) + '/' + 'random_blocks2_length200000_batches'+str(n_batches)
    if 'artificial' in dataset:
        dataroot += '_seqlen'+str(artificial_seq_len)+'_ratio'+str(linear_ratio_for_artificial_seq)
    for block_sz in [10, 100, 1000, 4000, 8000, 10000]:
        rs_rb2[(hidden_sz, block_sz)] = ld.ResultSet_1toM(
            dataroot,
            datapaths['1toM'][dataset]['FCL'+str(hidden_sz)]['random_blocks2'][(0.05, 8, 200000, block_sz)]
        )
        rs_rb2[(hidden_sz, block_sz)].load_analytics()
      
    
## Random split (random two-split) scenario combining two frequencies - Various lengths ## 

rs_rb2_2freq = {}

for hidden_sz in [10, 20, 60]:
    dataroot = project_root+'/Results/1toM/' + dataset + '/' + 'FCL'+str(hidden_sz) + '/' + 'random_blocks2_2freq_length200000_batches'+str(n_batches)
    if 'artificial' in dataset:
        dataroot += '_seqlen'+str(artificial_seq_len)+'_ratio'+str(linear_ratio_for_artificial_seq)
    for block_sz_couple in [(100, 1000), (100, 10000), (1000, 10000)]:
        rs_rb2_2freq[(hidden_sz, block_sz_couple[0], block_sz_couple[1])] = ld.ResultSet_1toM(
            dataroot,
            datapaths['1toM'][dataset]['FCL'+str(hidden_sz)]['random_blocks2_2freq'][(0.05, 8, 200000, block_sz_couple[0], block_sz_couple[1])]
        )
        rs_rb2_2freq[(hidden_sz, block_sz_couple[0], block_sz_couple[1])].load_analytics()


Loading analytics...
load_data set to False. Data sequences not loaded.
load_atc set to False. Autocorrelations not loaded.

Loading analytics...
load_data set to False. Data sequences not loaded.
load_atc set to False. Autocorrelations not loaded.

Loading analytics...
load_data set to False. Data sequences not loaded.
load_atc set to False. Autocorrelations not loaded.

Loading analytics...
load_data set to False. Data sequences not loaded.
load_atc set to False. Autocorrelations not loaded.

Loading analytics...
load_data set to False. Data sequences not loaded.
load_atc set to False. Autocorrelations not loaded.

Loading analytics...
load_data set to False. Data sequences not loaded.
load_atc set to False. Autocorrelations not loaded.

Loading analytics...
load_data set to False. Data sequences not loaded.
load_atc set to False. Autocorrelations not loaded.

Loading analytics...
load_data set to False. Data sequences not loaded.
load_atc set to False. Autocorrelations not loaded.


In [8]:
os.chdir(project_root+"/plots")

## Labels history

In [ ]:
rs_rb2[(20, 10)].lbl_history([0.0])

In [ ]:
rs_rb2[(20, 100)].lbl_history([0.0])

In [ ]:
rs_rb2[(20, 320)].lbl_history([0.0])

In [ ]:
rs_rb2[(20, 1000)].lbl_history([0.0])

In [ ]:
rs_rb2[(20, 2000)].lbl_history([0.0])

In [ ]:
rs_rb2[(20, 10000)].lbl_history([0.0])

## Accuracy = f(t) plots

In [ ]:
ld.get_acc(
    T_list = [0.0],
    acc_temp_orig = rs_rb2[(20, 100)].var_acc_orig,
    acc_temp_shuffled = {
        0.0: {block_sz: rs_rb2[(20, 100)].var_acc_shfl[0.0][block_sz] for block_sz in [1]}
    }
)

In [ ]:
ld.get_acc(
    T_list = [0.0],
    acc_temp_orig = rs_rb2[(20, 1000)].var_acc_orig,
    acc_temp_shuffled = {
        0.0: {block_sz: rs_rb2[(20, 1000)].var_acc_shfl[0.0][block_sz] for block_sz in [1]}
    }
)

In [ ]:
ld.get_acc(
    T_list = [0.0],
    acc_temp_orig = rs_rb2[(20, 10000)].var_acc_orig,
    acc_temp_shuffled = {
        0.0: {block_sz: rs_rb2[(20, 10000)].var_acc_shfl[0.0][block_sz] for block_sz in [1]}
    }
)

In [9]:
cf_unit = {}
cf = {}
cf_explr = {}

for hidden_sz in [5, 10, 20, 60, 200]:
    for block_sz in [10, 100, 1000, 4000, 8000, 10000]:
        cf_unit[(hidden_sz,block_sz)] = []
        n_seq = len(rs_rb2[(hidden_sz, block_sz)].var_acc_orig[0.0])
        cf_at_fullexplr = []
        for seq_id in range(n_seq):
            _cf, _t_explr = ld.get_cf(
                    rs_rb2[(hidden_sz,block_sz)].train_labels_orig[0.0][seq_id],
                    rs_rb2[(hidden_sz,block_sz)].var_acc_orig[0.0][seq_id][:,0],
                    rs_rb2[(hidden_sz,block_sz)].var_acc_shfl[0.0][1][0][:,0]
                )
            cf_unit[(hidden_sz,block_sz)].append(_cf)
            cf_at_fullexplr.append(_cf[_t_explr])
        cf_unit[(hidden_sz,block_sz)] = np.stack(cf_unit[(hidden_sz,block_sz)], axis=1)
        cf[(hidden_sz,block_sz)] = np.mean(cf_unit[(hidden_sz,block_sz)], axis=1)
        cf_explr[(hidden_sz,block_sz)] = np.mean(np.array(cf_at_fullexplr))
    
    if hidden_sz in [10, 20, 60]:
        for block_sz_couple in [(100, 1000), (100, 10000), (1000, 10000)]:
            cf_unit[(hidden_sz, block_sz_couple[0], block_sz_couple[1])] = []
            n_seq = len(rs_rb2_2freq[(hidden_sz, block_sz_couple[0], block_sz_couple[1])].var_acc_orig[0.0])
            for seq_id in range(n_seq):
                cf_unit[(hidden_sz, block_sz_couple[0], block_sz_couple[1])].append(
                    ld.get_cf(
                        rs_rb2_2freq[(hidden_sz, block_sz_couple[0], block_sz_couple[1])].train_labels_orig[0.0][seq_id],
                        rs_rb2_2freq[(hidden_sz, block_sz_couple[0], block_sz_couple[1])].var_acc_orig[0.0][seq_id][:,0],
                        rs_rb2_2freq[(hidden_sz, block_sz_couple[0], block_sz_couple[1])].var_acc_shfl[0.0][1][0][:,0]
                    )
                )
            cf_unit[(hidden_sz, block_sz_couple[0], block_sz_couple[1])] = np.stack(cf_unit[(hidden_sz, block_sz_couple[0], block_sz_couple[1])], axis=1)
            cf_unit[(hidden_sz, block_sz_couple[0], block_sz_couple[1])] = cf_unit[(hidden_sz, block_sz_couple[0], block_sz_couple[1])][0,:]
            cf[(hidden_sz, block_sz_couple[0], block_sz_couple[1])] = np.mean(cf_unit[(hidden_sz, block_sz_couple[0], block_sz_couple[1])])

In [ ]:
fig = plt.figure(figsize=(15,10))
cf_ax = plt.subplot(111)
for block_sz in [10, 100, 1000, 10000]:
    cf_ax.plot(
        cf[(20, block_sz)],
        label='Forgetting score - Split size {0:d}'.format(block_sz)
    )
cf_ax.legend(loc='upper center', bbox_to_anchor=(0.5, 0.15),
					  fancybox=True, shadow=True, ncol=2,
					  prop={'size': 16})
plt.title("Catastrophic forgetting on split sequences as a function of split sequence length", fontsize = 14)
plt.xlabel('CF score', fontsize=14)
plt.ylabel('Iterations', fontsize=14)

In [ ]:
fig = plt.figure(figsize=(15,10))
cf_ax = plt.subplot(111)

for block_sz in [1000, 10000]:
    cf_ax.plot(
        cf[(20, block_sz)],
        label='Forgetting score - Split size {0:d}'.format(block_sz)
    )

for block_sz_couple in [(1000, 10000)]:
    cf_ax.plot(
        cf[(20, block_sz_couple[0], block_sz_couple[1])],
        label='Forgetting score - Split size {0:d} x {1:d}'.format(block_sz_couple[0], block_sz_couple[1])
    )

cf_ax.legend(loc='upper center', bbox_to_anchor=(0.5, 0.15),
					  fancybox=True, shadow=True, ncol=2,
					  prop={'size': 16})
plt.title("Catastrophic forgetting on split sequences as a function of split sequence length", fontsize = 14)
plt.ylabel('CF score', fontsize=14)
plt.xlabel('Iterations', fontsize=14)

n_tests=200
seq_length=200000
xtick_scale = 25
xtick_pos = xtick_scale*np.arange((n_tests//xtick_scale)+1)
xtick_labels = int(seq_length/((n_tests//xtick_scale)))*np.arange((n_tests//xtick_scale)+1)
plt.xticks(xtick_pos, xtick_labels)

Next let's try to obtain a plot of mean CF score as a function of (hidden_sz, block_sz) for single-frequency signals:

In [10]:
fig = plt.figure(figsize=(15,12))
cf_ax = plt.axes(projection='3d')

hidden_sizes = [5, 10, 20, 60, 200]
block_sizes = [10, 100, 1000, 4000, 8000, 10000]

cf_avg = {}

for hidden_sz in hidden_sizes:
    for block_sz in block_sizes:
        # cf_avg[(hidden_sz, block_sz)] = np.mean(cf[(hidden_sz, block_sz)])
        cf_avg[(hidden_sz, block_sz)] = cf_explr[(hidden_sz,block_sz)]
    
    cf_avg_arr = np.array([cf_avg[(hidden_sz, bs)] for bs in block_sizes])
    cf_ax.plot(
        hidden_sz*np.ones(len(block_sizes)),
        #np.log10(np.array(block_sizes)),
        np.array(block_sizes),
        cf_avg_arr
    )
    
cf_ax.set_xlabel('Hidden layer size', fontsize=14)
cf_ax.set_ylabel('Timescale of sequence', fontsize=14)
cf_ax.set_zlabel('Catastrophic Forgetting score', fontsize=14)

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

Text(0.5, 0, 'Catastrophic Forgetting score')

## Autocorrelation plots (computed a posteriori)

Let's plot the autocorrelation function to DARPA standards:

In [ ]:
rs.get_atc(
    T_list=[0.4],
    n_tests=10,
    out_filename='atc_artificial32_T04'
)